In [ ]:
import cv2
import numpy as np

In [ ]:
def brightness_contrast(img):
    global bright , contrast
    bright = 255
    contrast = 127

    def nothing(x):
        pass

    def apply_brightness_contrast(input_img, brightness = 255, contrast = 127):
        brightness = map(brightness, 0, 510, -255, 255)
        contrast = map(contrast, 0, 254, -127, 127)

        if brightness != 0:
            if brightness > 0:
                shadow = brightness
                highlight = 255
            else:
                shadow = 0
                highlight = 255 + brightness
            alpha_b = (highlight - shadow)/255
            gamma_b = shadow

            buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
        else:
            buf = input_img.copy()

        if contrast != 0:
            f = float(131 * (contrast + 127)) / (127 * (131 - contrast))
            alpha_c = f
            gamma_c = 127*(1-f)

            buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

        return buf

    def map(x, in_min, in_max, out_min, out_max):
        return int((x-in_min) * (out_max-out_min) / (in_max-in_min) + out_min)

    
    
    cv2.namedWindow('img',1)
    cv2.createTrackbar('bright', 'img', bright, 2*255, nothing)
    cv2.createTrackbar('contrast', 'img', contrast, 2*127, nothing)
    img2 = None
    while True:
        img1=img.copy() 
        bright = cv2.getTrackbarPos('bright', 'img')
        contrast = cv2.getTrackbarPos('contrast', 'img')
        img1 = apply_brightness_contrast(img1,bright,contrast)
        cv2.imshow("img", img1)
        #cv2.imshow("img1", img1)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            img2 = img1
            break
    cv2.destroyAllWindows()
    return img2

In [ ]:
def color_segmentation(img): 
    def nothing(x):
        pass

    cv2.namedWindow('image')

    cv2.createTrackbar('lowH','image',0,179,nothing)
    cv2.createTrackbar('highH','image',179,179,nothing)

    cv2.createTrackbar('lowS','image',0,255,nothing)
    cv2.createTrackbar('highS','image',255,255,nothing)

    cv2.createTrackbar('lowV','image',0,255,nothing)
    cv2.createTrackbar('highV','image',255,255,nothing)
    lower_hsv = None
    higher_hsv = None
    while(True):
        frame = img.copy()

        ilowH = cv2.getTrackbarPos('lowH', 'image')
        ihighH = cv2.getTrackbarPos('highH', 'image')
        ilowS = cv2.getTrackbarPos('lowS', 'image')
        ihighS = cv2.getTrackbarPos('highS', 'image')
        ilowV = cv2.getTrackbarPos('lowV', 'image')
        ihighV = cv2.getTrackbarPos('highV', 'image')

        # convert color to hsv because it is easy to track colors in this color model
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_hsv = np.array([ilowH, ilowS, ilowV])
        higher_hsv = np.array([ihighH, ihighS, ihighV])
        # Apply the cv2.inrange method to create a mask
        mask = cv2.inRange(hsv, lower_hsv, higher_hsv)
        # Apply the mask on the image to extract the original color
        frame = cv2.bitwise_and(frame, frame, mask=mask)
        cv2.imshow('image', frame)
        cv2.imshow('img', frame)
        # Press q to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return lower_hsv , higher_hsv

In [ ]:
def color_overlay(img): 
    def nothing(x):
        pass
    def verify_aplha_channel(img):
        try :
            img.shape[3]
        except IndexError:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
        return img
    def apply_color_overlay(img , intensity=0.5 , blue=0 , green=0 , red=0):
        img = verify_aplha_channel(img)
        h , w , c = img.shape
        color_bgra = (blue , green , red , 1)
        overlay = np.full((h,w,c) , color_bgra , dtype = 'uint8')
        img = cv2.addWeighted(overlay , intensity , img , 1.0 , 0)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        return img
    
    cv2.namedWindow('image')

    cv2.createTrackbar('blue','image',0,255,nothing)
    cv2.createTrackbar('green','image',0,255,nothing)
    cv2.createTrackbar('red','image',0,255,nothing)
    cv2.createTrackbar('intensity%','image',0,100,nothing)
    img2 = None
    while(True):
        frame = img.copy()

        blue = cv2.getTrackbarPos('blue', 'image')
        green = cv2.getTrackbarPos('green', 'image')
        red = cv2.getTrackbarPos('red', 'image')
        intensity = cv2.getTrackbarPos('intensity%', 'image')/100.0
        frame = apply_color_overlay(frame , intensity , blue , green , red)
        cv2.imshow('image', frame)
        cv2.imshow('img', frame)
        # Press q to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            img2  = frame
            break

    cv2.destroyAllWindows()
    return img2

In [ ]:
def add_noise_blur(img):
    row,col,ch= img.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = np.array(gauss.reshape(row,col,ch) , np.uint8)
    noisy = cv2.addWeighted(img , 1 , gauss,0.4,0)
    cv2.imshow('noisy' , cv2.GaussianBlur(noisy , (7,7) , 0))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return noisy

In [ ]:
img = cv2.imread('abc/1.jpg')
img = cv2.resize(img , (750,500))
imgbc = brightness_contrast(img)
lower_range , higher_range = color_segmentation(imgbc)
print(lower_range , higher_range)

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#range_min = np.array([9, 0, 50])
#range_max =  np.array([30, 181, 79])
mask = cv2.inRange(hsv, lower_range, higher_range)
mask  = cv2.morphologyEx(cv2.medianBlur(mask , 3) , cv2.MORPH_CLOSE, np.ones((3,3) , np.uint8))
res = cv2.bitwise_and(img , img , mask = mask)
dst = cv2.inpaint(img , mask , 3 , cv2.INPAINT_TELEA)
cv2.imshow("inpaint", dst)
cv2.imshow("mask", mask)
cv2.imshow("spots", res)
cv2.imshow("original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite('mask_1.jpg' , mask)
#cv2.imwrite('res_1.jpg' , res)
cv2.imwrite('mask.jpg' , mask)
cv2.imwrite('img.jpg' , img)

In [ ]:
imgc = cv2.imread('2.jpg')
imgc = cv2.resize(imgc , (750,500))
imgco_c = color_overlay(imgc)
imgbc_c = brightness_contrast(imgco_c)
#imgbn_c = add_noise_blur(imgbc_c)
img1 = img.copy()
mask_inv = cv2.bitwise_not(mask)
img_bg = cv2.bitwise_and(imgbc_c , imgbc_c , mask = mask_inv)
img_fg = cv2.bitwise_and(img1 , img1 , mask = mask)
img_new = cv2.add(img_bg , res)
cv2.imshow("mask_inv", mask_inv)
cv2.imshow("mask", mask)
cv2.imshow("img_new", img_new)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow("img", img_new)
kernel = np.ones((5,5) , np.uint8)
#cv2.imshow('blur1' , cv2.medianBlur(img_new , 7))
#cv2.imshow('blur2' , cv2.GaussianBlur(img_new , (5,5) , 0))
#cv2.imshow('blur3' , cv2.bilateralFilter(img_new , 15 , 15 , 75))
#cv2.imshow('open' , cv2.morphologyEx(img_new , cv2.MORPH_OPEN, kernel))
#cv2.imshow('close' , cv2.morphologyEx(cv2.medianBlur(img_new , 9) , cv2.MORPH_CLOSE, np.ones((3,3) , np.uint8)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite('res3.jpg' , img_new)